In [15]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re

from lib import data_utils 

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLpath = datapath + 'AnnotatedPLData/PLCoref'
CENpath = datapath + 'AnnotatedCENData/CENCoref'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'

#import data

#create empty data frame
PL_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts into one dataframe
for n in range(1,3):

    #get story path
    storypath = PLpath+'/story' + str(n) + '.txt'

    #read in story
    corpus = data_utils.read_story(storypath)

    #list of features
    features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    for f in features:
        #empty list
        feat = []
        with open(PLInter + f + 'FeatureBoolean'+'/Story' + str(n) + '.txt', 'r') as doc:
            for line in doc:
                feat.append(eval(line.rstrip()))
    
        corpus[f] = feat

    #append to dataframe
    PL_data = pd.concat([PL_data, corpus], ignore_index=True)

PL_data

,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL
0,story1,1,1,"[ dragon , he , he , the dragon , He , hi...",dragon,dragon,43,2.499540
1,story1,1,1,"[ princess , the tsar 's daughter , her , h...",princess,princess,23,0.990763
2,story1,1,1,"[ tsar , tsar , father , tsar , her father...",tsar,tsar,9,-0.065380
3,story1,0,1,"[ princess' dog , a little dog that had follo...",princess' dog,dog,4,-0.442574
4,story1,1,1,"[ tsarina , mother , tsarina , tsarina ]",tsarina,tsarina,4,-0.442574
5,story1,0,1,"[ who is stronger , who in this world was str...",who is stronger,stronger,2,-0.593452
6,story1,0,1,"[ dragon's statement , a tanner in the city o...",dragon's statement,statement,3,-0.518013
7,story1,1,1,"[ Nikita , a tanner in the city of Kiev , Ni...",Nikita,Nikita,39,2.197784
8,story1,0,1,"[ tsar's coming , the tsar went in person to ...",tsar's coming,coming,3,-0.518013
9,story1,0,1,"[ his hands , his hands , his hands ]",his hands,hands,3,-0.518013


In [18]:
features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

for f in features:
    print(f)

CN
Dep
NER
SS
Triple
WN


In [3]:
from allennlp.predictors.predictor import Predictor
from allennlp_models import pretrained
import allennlp_models.tagging
# print(pretrained.get_pretrained_models())


# predictor_ss = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
predictor_ner = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo-2021.02.12.tar.gz")

results = predictor_ner.predict_instance("Did Uriah honestly think he could beat The Legend of Zelda in less than three hours?")


KeyboardInterrupt: 